# Project 2 Code
- all final code that goes into the report will be pulled from here

**To Do:**
1. The prices are still a little mysterious. When I convert the currency to usd, sometimes the number looks right, but sometimes it doesn't... We could either make an assumption that everything is in USD (which is prob unlikely) or we could try to dig a little deeper
2. I broadly categorized the amenities to be tv	wifi	chromecast	refrigerator	kitchen	view	parking	workspace. Please feel free to add more!
3. I changed the "New" rating to be -1 so that we could do some math manipulation later. I think we need to make an assumption later on what the placeholder is going to be for those listings.
4. If you guys find more thing that we need to do, then please add them to the summary table below on "Summary of things To Do and fix"

### Import python packages and data

In [30]:
import pandas as pd
import numpy as np
import pprint
! pip install CurrencyConverter
from currency_converter import CurrencyConverter
import requests

In [31]:
airbnb = pd.read_csv('airbnb.csv')
country_to_letters = pd.read_csv('country-code-to-currency-code-mapping.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

#pd.set_option('display.max_rows', None)

### Data cleaning

#### Summary of things To Do and fix
| Column | Number of non null values | Original Type | Type to Change to | To Do |
| ------ | ------------------------- | ------------- | ----------------- | ----- |
| Unnamed: 0    | 12805 |  int64   | | DROP |
| id            | 12805 |  int64   | |  |
| name          | 12805 |  object  | str | DROP after matching id and name |
| rating        | 12805 |  object  | str | * mainly float values, but when there are no rating, says "new" </br> * rating does not go below 3.0|
| reviews       | 12805 |  object  | int | * change type to int to make reviews easy to work with |
| host_name     | 12797 |  object  | | DROP after matching host name and host id |
| host_id       | 12805 |  float64 | int | |
| address       | 12805 |  object  | |  |
| features      | 12805 |  object  | | DROP since we have a summary of the features under other columns (toiles, bedrooms, studios, checkin, checkout) |
| amenities     | 12805 |  object  | | values are strung together with commas (,). when analyzing, we need to parse the strings |
| safety_rules  | 12805 |  object  | | values are strung together with commas (,). when analyzing, we need to parse the strings |
| hourse_rules  | 12805 |  object  | | * values are strung together with commas (,). when analyzing, we need to parse the strings </br> * rename column from hourse_rul to house_rule |
| img_links     | 12805 |  object  | | potentially use this column to see if the images help with reviews, but last in priority|
| price         | 12805 |  int64   | float | * Need to find out if the prices are recorded as US dollars  </br> * currently recroded as int, but investigate how it's being recorded|
| country       | 12805 |  object  | str | * Values have whitespaces, should remove |
| bathrooms     | 12805 |  int64   | | |
| beds          | 12805 |  int64   | | |
| guests        | 12805 |  int64   | | |
| toiles        | 12805 |  int64   | | * need to rename column|
| bedrooms      | 12805 |  int64   | | |
| studios       | 12805 |  int64   | | |
| checkin       | 12005 |  object  | | |
| checkout      | 10355 |  object  | datetime | * change to datetime </br> * figure out what to do with the null values|

In [32]:
# 1. Rename all the columns that are mispelled
column_rename = {
  'hourse_rules': 'house_rules',
  'toiles': 'toilets'
}
airbnb = airbnb.rename(columns=column_rename)
airbnb.head()

,Unnamed: 0,id,name,rating,reviews,host_name,host_id,address,features,amenities,safety_rules,house_rules,img_links,price,country,bathrooms,beds,guests,toilets,bedrooms,studios,checkin,checkout
0,0,49849504,Perla bungalov,4.71,64,Mehmetcan,357334205.00,"Kartepe, Kocaeli, Turkey","2 guests,2 bedrooms,1 bed,1 bathroom","Mountain view,Valley view,Lake access,Kitchen,...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: Flexible,Check out: 12:00 pm,Pets ar...",https://a0.muscache.com/im/pictures/a5da5cb7-c...,8078,Turkey,1,1,2,0,2,0,Flexible,12 00 pm
1,1,50891766,Authentic Beach Architect Sheltered Villa with...,New,0,Fatih,386223873.00,"Kaş, Antalya, Turkey","4 guests,2 bedrooms,2 beds,2 bathrooms","Kitchen,Wifi,Dedicated workspace,Free parking ...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 4:00 pm - 11:00 pm,Check out: 10:00 ...",https://a0.muscache.com/im/pictures/61b70855-2...,4665,Turkey,2,2,4,0,2,0,4 00 pm - 11 00 pm,10 00 am
2,2,50699164,cottages sataplia,4.85,68,Giorgi,409690853.00,"Imereti, Georgia","4 guests,1 bedroom,3 beds,1 bathroom","Mountain view,Kitchen,Wifi,Dedicated workspace...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 1:00 pm,Check out: 12:00 pm,Se...",https://a0.muscache.com/im/pictures/miso/Hosti...,5991,Georgia,1,3,4,0,1,0,After 1 00 pm,12 00 pm
3,3,49871422,Sapanca Breathable Bungalow,5.0,13,Melih,401873242.00,"Sapanca, Sakarya, Turkey","4 guests,1 bedroom,2 beds,1 bathroom","Mountain view,Valley view,Kitchen,Wifi,Free pa...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/72e6396e-e...,11339,Turkey,1,2,4,0,1,0,After 2 00 pm,12 00 pm
4,4,51245886,Bungalov Ev 2,New,0,Arp Sapanca,414884116.00,"Sapanca, Sakarya, Turkey","2 guests,1 bedroom,1 bed,1 bathroom","Kitchen,Wifi,Free parking on premises,TV,Air c...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/73973308-e...,6673,Turkey,1,1,2,0,1,0,After 2 00 pm,12 00 pm


In [33]:
# 2. Change all datatype to its appropriate type

print("Before changing the types of column")
# replace all commas on "reviews" column
airbnb.reviews.replace({",":""}, inplace=True, regex=True)
# replace all New ratings as -1 to convert column to floats
airbnb.rating.replace({"New":"-1"}, inplace=True, regex=True)

print(airbnb.dtypes)

col_dtypes = {
  'name': 'string',
  'rating': 'float64',
  'reviews': 'int64',
  'host_id': 'int64',
  'price': 'float64',
  'country': 'string',
  'checkin': 'string'
}
airbnb = airbnb.astype(col_dtypes, errors = 'ignore')
# airbnb['checkout'] = pd.to_datetime(airbnb['checkout'], errors = 'coerce')
print("\nAfter changing the types of column")
airbnb.dtypes
# airbnb

Before changing the types of column
Unnamed: 0        int64
id                int64
name             object
rating           object
reviews          object
host_name        object
host_id         float64
address          object
features         object
amenities        object
safety_rules     object
house_rules      object
img_links        object
price             int64
country          object
bathrooms         int64
beds              int64
guests            int64
toilets           int64
bedrooms          int64
studios           int64
checkin          object
checkout         object
dtype: object

After changing the types of column


Unnamed: 0        int64
id                int64
name             string
rating          float64
reviews           int64
host_name        object
host_id           int64
address          object
features         object
amenities        object
safety_rules     object
house_rules      object
img_links        object
price           float64
country          string
bathrooms         int64
beds              int64
guests            int64
toilets           int64
bedrooms          int64
studios           int64
checkin          string
checkout         object
dtype: object

#### Assumption 1: The Prices are in country's currency
3. Use 3rd party api to convert the money
* pycountry and currentcurrency package to find out foreign currency to usd


In [34]:
# redefine country to match all.csv country name

# Unique country names from all_countries
all_countries_set = set(airbnb['country'].str.strip())
# Unique country names from country_to_letters
country_to_letters_set = set(country_to_letters['Country'])
missing_countries = all_countries_set - country_to_letters_set
pprint.pp(missing_countries)

# # change the values of country to match the all.csv file
country_change = {
  'Australia': 'Australian',
  'Bosnia & Herzegovina': 'Bosnia and Herzegovina',
  'Brunei': 'Brunei Darussalam',
  'Congo': 'Congo Republic of the Democratic',
  'Croatia': 'Croatia (Hrvatska)',
  'Czechia': 'Czech Republic',
  # kosovo could be equivalent to germany bc they both use euro
  'Kosovo': 'Germany',
  'Slovakia': 'Slovakia (Slovak Republic)',
  'South Korea': 'Korea South',
  'St Martin': 'Saint Martin (French part)',
  'Svalbard & Jan Mayen': 'Svalbard and Jan Mayen Islands',
  # Åland Islands could be equivalent to germany bc they both use euro
  'Åland Islands': 'Germany'}

airbnb.country.replace(country_change, inplace=True, regex=True)

{'Australia',
 'Bosnia & Herzegovina',
 'Brunei',
 'Congo',
 'Croatia',
 'Czechia',
 'Kosovo',
 'Slovakia',
 'South Korea',
 'St Martin',
 'Svalbard & Jan Mayen',
 'Åland Islands'}


In [35]:
# Function to convert price to USD
def convert_price_to_usd(row, converter):
    try:
        # remove whitespace
        country_clean = row['country'].strip()
        # locate the country on country code to currency code df and find 3 digit currency code
        country_code_series = country_to_letters.loc[country_to_letters['Country'] == country_clean]['Code']
        # should have one currency code
        country_code = country_code_series.values[0]
        # change from country code price to usd
        usd_price = converter.convert(row['price'], country_code, 'USD')
        return usd_price
    # if any error happens print and return none
    except Exception as e:
        print(f"Error converting price for {row['country']}: {e}")
        return None

# Initialize the converter
converter = CurrencyConverter()

# Apply the conversion function to the DataFrame
airbnb['usd_price'] = airbnb.apply(convert_price_to_usd, axis=1, converter=converter)

Error converting price for  Georgia: GEL is not a supported currency
Error converting price for  Vietnam: VND is not a supported currency
Error converting price for  Georgia: GEL is not a supported currency
Error converting price for  Georgia: GEL is not a supported currency
Error converting price for  Lebanon: LBP is not a supported currency
Error converting price for  Taiwan: TWD is not a supported currency
Error converting price for  Georgia: GEL is not a supported currency
Error converting price for  Vietnam: VND is not a supported currency
Error converting price for  Georgia: GEL is not a supported currency
Error converting price for  Georgia: GEL is not a supported currency
Error converting price for  Vietnam: VND is not a supported currency
Error converting price for  Armenia: AMD is not a supported currency
Error converting price for  Georgia: GEL is not a supported currency
Error converting price for  Taiwan: TWD is not a supported currency
Error converting price for  Vietnam:

In [36]:
airbnb.head()

,Unnamed: 0,id,name,rating,reviews,host_name,host_id,address,features,amenities,safety_rules,house_rules,img_links,price,country,bathrooms,beds,guests,toilets,bedrooms,studios,checkin,checkout,usd_price
0,0,49849504,Perla bungalov,4.71,64,Mehmetcan,357334205,"Kartepe, Kocaeli, Turkey","2 guests,2 bedrooms,1 bed,1 bathroom","Mountain view,Valley view,Lake access,Kitchen,...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: Flexible,Check out: 12:00 pm,Pets ar...",https://a0.muscache.com/im/pictures/a5da5cb7-c...,8078.00,Turkey,1,1,2,0,2,0,Flexible,12 00 pm,244.14
1,1,50891766,Authentic Beach Architect Sheltered Villa with...,-1.00,0,Fatih,386223873,"Kaş, Antalya, Turkey","4 guests,2 bedrooms,2 beds,2 bathrooms","Kitchen,Wifi,Dedicated workspace,Free parking ...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 4:00 pm - 11:00 pm,Check out: 10:00 ...",https://a0.muscache.com/im/pictures/61b70855-2...,4665.00,Turkey,2,2,4,0,2,0,4 00 pm - 11 00 pm,10 00 am,140.99
2,2,50699164,cottages sataplia,4.85,68,Giorgi,409690853,"Imereti, Georgia","4 guests,1 bedroom,3 beds,1 bathroom","Mountain view,Kitchen,Wifi,Dedicated workspace...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 1:00 pm,Check out: 12:00 pm,Se...",https://a0.muscache.com/im/pictures/miso/Hosti...,5991.00,Georgia,1,3,4,0,1,0,After 1 00 pm,12 00 pm,NaN
3,3,49871422,Sapanca Breathable Bungalow,5.00,13,Melih,401873242,"Sapanca, Sakarya, Turkey","4 guests,1 bedroom,2 beds,1 bathroom","Mountain view,Valley view,Kitchen,Wifi,Free pa...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/72e6396e-e...,11339.00,Turkey,1,2,4,0,1,0,After 2 00 pm,12 00 pm,342.70
4,4,51245886,Bungalov Ev 2,-1.00,0,Arp Sapanca,414884116,"Sapanca, Sakarya, Turkey","2 guests,1 bedroom,1 bed,1 bathroom","Kitchen,Wifi,Free parking on premises,TV,Air c...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/73973308-e...,6673.00,Turkey,1,1,2,0,1,0,After 2 00 pm,12 00 pm,201.68


In [37]:
countries_with_nan_usd_price = airbnb[airbnb['usd_price'].isna()]['country'].unique()
countries_with_nan_usd_price

<StringArray>
[                         ' Georgia',                          ' Vietnam',
                          ' Lebanon',                           ' Taiwan',
                          ' Armenia',                          ' Ukraine',
                          ' Albania',                         ' Tanzania',
                        ' Sri Lanka',           ' Bosnia and Herzegovina',
                            ' Kenya',                           ' Serbia',
                       ' Seychelles',                          ' Morocco',
                            ' Egypt',             ' United Arab Emirates',
                         ' Pakistan',                            ' Nepal',
                         'Sri Lanka',                         ' Cambodia',
                       ' Azerbaijan',                       ' Costa Rica',
                           ' Panama',                             ' Peru',
                            ' Chile',                           ' Belize',
           

In [38]:
def get_exchange_rates(base_currency='USD'):
    url = 'https://api.exchangerate-api.com/v4/latest/' + base_currency
    response = requests.get(url)
    data = response.json()
    return data['rates']

# Fetch exchange rates
exchange_rates = get_exchange_rates()

# Define currency codes for the countries of interest
currency_codes = {
    'Georgia': 'GEL',
    'Vietnam': 'VND',
    'Lebanon': 'LBP',
    'Taiwan': 'TWD',
    'Armenia': 'AMD',
    'Ukraine': 'UAH',
    'Albania': 'ALL',
    'Tanzania': 'TZS',
    'Sri Lanka': 'LKR',
    'Bosnia and Herzegovina': 'BAM',
    'Kenya': 'KES',
    'Serbia': 'RSD',
    'Seychelles': 'SCR',
    'Morocco': 'MAD',
    'Egypt': 'EGP',
    'United Arab Emirates': 'AED',
    'Pakistan': 'PKR',
    'Nepal': 'NPR',
    'Cambodia': 'KHR',
    'Azerbaijan': 'AZN',
    'Costa Rica': 'CRC',
    'Panama': 'PAB',
    'Peru': 'PEN',
    'Chile': 'CLP',
    'Belize': 'BZD',
    'Colombia': 'COP',
    'Bolivia': 'BOB',
    'Mauritius': 'MUR',
    'Fiji': 'FJD',
    'Maldives': 'MVR',
    'Myanmar': 'MMK',
    'Brunei Darussalam': 'BND',
    'Kyrgyzstan': 'KGS',
    'Uruguay': 'UYU',
    'Argentina': 'ARS',
    'Bahamas': 'BSD',
    'Oman': 'OMR',
    'Kuwait': 'KWD',
    'Saudi Arabia': 'SAR',
    'Kazakhstan': 'KZT',
    'Uganda': 'UGX',
    'Qatar': 'QAR',
    'Jordan': 'JOD',
    'Bangladesh': 'BDT',
    'Mongolia': 'MNT',
    'Uzbekistan': 'UZS',
    'Rwanda': 'RWF',
    'Honduras': 'HNL',
    'Nicaragua': 'NIO',
    'Vanuatu': 'VUV',
    'Cayman Islands': 'KYD',
    'French Polynesia': 'XPF',
    'Jamaica': 'JMD',
    'Madagascar': 'MGF',
    'Tunisia': 'TND',
    'Congo Republic of the Democratic': 'CDF',
    'Cuba': 'CUP',
    'Nigeria': 'NGN',
    'Saint Martin (French part)': 'EUR',
    'Guatemala': 'GTQ',
    'Senegal': 'XOF',
    'Namibia': 'NAD'
}

# Convert to USD
conversion_rates = {}
for country, currency in currency_codes.items():
    if currency in exchange_rates:
        conversion_rates[country] = 1/exchange_rates[currency]
    else:
        conversion_rates[country] = None
        print(f"Currency rate for {currency} not found.")

print(conversion_rates)
# manually looked the conversion and set currency conversion rate
conversion_rates['Madagascar'] = 0.0000440133

Currency rate for MGF not found.
{'Georgia': 0.3676470588235294, 'Vietnam': 3.954365045627441e-05, 'Lebanon': 1.11731843575419e-05, 'Taiwan': 0.03045994517209869, 'Armenia': 0.002577984016499098, 'Ukraine': 0.024372410431391663, 'Albania': 0.010790978741771878, 'Tanzania': 0.0003738597278301181, 'Sri Lanka': 0.0033004389583814646, 'Bosnia and Herzegovina': 0.5524861878453039, 'Kenya': 0.007649353629618298, 'Serbia': 0.00925154963456379, 'Seychelles': 0.07288629737609328, 'Morocco': 0.10101010101010101, 'Egypt': 0.020695364238410598, 'United Arab Emirates': 0.2724795640326976, 'Pakistan': 0.003591954022988506, 'Nepal': 0.0074610161904051335, 'Cambodia': 0.00024226214702405177, 'Azerbaijan': 0.5882352941176471, 'Costa Rica': 0.0018919327985469957, 'Panama': 1.0, 'Peru': 0.267379679144385, 'Chile': 0.001048020289672808, 'Belize': 0.5, 'Colombia': 0.00024845584691144534, 'Bolivia': 0.14450867052023122, 'Mauritius': 0.021486892995272885, 'Fiji': 0.4424778761061947, 'Maldives': 0.06472491909

In [39]:
# execute the conversion
def convert_curr(row):
  country = row['country'].strip()
  # if country is that of counversion rates dictionary, convert
  if country in conversion_rates:
    return row.price * conversion_rates[country]
  # else leave usd price as is (bc already coverted)
  else:
    return row.usd_price

airbnb['usd_price'] = airbnb.apply(lambda row: convert_curr(row), axis=1)
airbnb.head()

,Unnamed: 0,id,name,rating,reviews,host_name,host_id,address,features,amenities,safety_rules,house_rules,img_links,price,country,bathrooms,beds,guests,toilets,bedrooms,studios,checkin,checkout,usd_price
0,0,49849504,Perla bungalov,4.71,64,Mehmetcan,357334205,"Kartepe, Kocaeli, Turkey","2 guests,2 bedrooms,1 bed,1 bathroom","Mountain view,Valley view,Lake access,Kitchen,...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: Flexible,Check out: 12:00 pm,Pets ar...",https://a0.muscache.com/im/pictures/a5da5cb7-c...,8078.00,Turkey,1,1,2,0,2,0,Flexible,12 00 pm,244.14
1,1,50891766,Authentic Beach Architect Sheltered Villa with...,-1.00,0,Fatih,386223873,"Kaş, Antalya, Turkey","4 guests,2 bedrooms,2 beds,2 bathrooms","Kitchen,Wifi,Dedicated workspace,Free parking ...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 4:00 pm - 11:00 pm,Check out: 10:00 ...",https://a0.muscache.com/im/pictures/61b70855-2...,4665.00,Turkey,2,2,4,0,2,0,4 00 pm - 11 00 pm,10 00 am,140.99
2,2,50699164,cottages sataplia,4.85,68,Giorgi,409690853,"Imereti, Georgia","4 guests,1 bedroom,3 beds,1 bathroom","Mountain view,Kitchen,Wifi,Dedicated workspace...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 1:00 pm,Check out: 12:00 pm,Se...",https://a0.muscache.com/im/pictures/miso/Hosti...,5991.00,Georgia,1,3,4,0,1,0,After 1 00 pm,12 00 pm,2202.57
3,3,49871422,Sapanca Breathable Bungalow,5.00,13,Melih,401873242,"Sapanca, Sakarya, Turkey","4 guests,1 bedroom,2 beds,1 bathroom","Mountain view,Valley view,Kitchen,Wifi,Free pa...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/72e6396e-e...,11339.00,Turkey,1,2,4,0,1,0,After 2 00 pm,12 00 pm,342.70
4,4,51245886,Bungalov Ev 2,-1.00,0,Arp Sapanca,414884116,"Sapanca, Sakarya, Turkey","2 guests,1 bedroom,1 bed,1 bathroom","Kitchen,Wifi,Free parking on premises,TV,Air c...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/73973308-e...,6673.00,Turkey,1,1,2,0,1,0,After 2 00 pm,12 00 pm,201.68


No conclusion on the price column. Can't tell whehter it's because of wrong decimal places or in foreign currency.
I converted from foreign currency to US dollars, but USD looks correct for some but not all. I think we need to investigate this column a little bit more...

jon: thanks Sarah! Ravi and I managed to find some listings and the prices seem quite similar to those in the local currency... I think we can probably just caveat that it's in local currency? What are some examples of rows that didn't seem right in USD?

#### Creating columns of different amenities we want to study

In [40]:
# create a sorted list of unique amenities so we can see what are the key amenities we want to look at
unique_amenities = sorted(list(airbnb['amenities'].str.split(',').explode().unique()))
unique_amenities

[' 2–5 years old',
 ' 2–5 years old and 5–10 years old',
 ' 5–10 years old and 10+ years old',
 ' Amazon Prime Video',
 ' Apple TV',
 ' Chromecast',
 ' DVD player',
 ' Disney+',
 ' Fire TV',
 ' HBO Max',
 ' Hulu',
 ' Netflix',
 ' Roku',
 ' cable/satellite TV',
 ' gas powered fridge. refrigerator',
 ' heated',
 ' infinity',
 ' lap pool',
 ' olympic-sized',
 ' open 24 hours',
 ' open specific hours',
 ' rooftop',
 ' saltwater',
 ' standard cable/satellite',
 ' sun loungers',
 ' wood-burning',
 '100" HDTV with Amazon Prime Video',
 '100" TV',
 '100" TV with Chromecast',
 '100" TV with Netflix',
 '101" HDTV with HBO Max',
 '107" HDTV with standard cable/satellite',
 '108" HDTV with standard cable/satellite',
 '12" TV with Amazon Prime Video',
 '12" TV with Netflix',
 '12" TV with standard cable/satellite',
 '12v cool box refrigerator',
 '14" HDTV with cable/satellite TV',
 '14" TV',
 '15" TV',
 '15" TV with standard cable/satellite',
 '16" TV',
 '165" HDTV with cable/satellite TV',
 '17" T

In [41]:
# First, we want to make another column that holds the lower case list of all amenities
airbnb['amenities_list'] = airbnb['amenities'].apply(lambda x: [item.lower().strip() for item in x.split(',')])

# I'm adding keywords that could help us find the 'common amenities'
# each of these keys will be another column and the value will be either yes or no (depending if listing contains these keywords)
# please feel free to add more amenities categories
# we could also do the same thing for safety rules if we want to analyze that as well
categories = {
    'tv': ['tv'],
    'wifi': ['wifi'],
    'chromecast': ['chromecast'],
    'refrigerator': ['refrigerator', 'fridge'],
    'kitchen': ['kitchen'],
    'view': ['view'],
    'parking': ['parking'],
    'workspace': ['workspace'],
    'pool': ['pool'],
    'gym': ['gym'],    
}

# Initialize columns with "no"
for category in categories:
    airbnb[category] = False

# Update columns with "yes" where the amenity is present
for category, keywords in categories.items():
    airbnb[category] = airbnb['amenities_list'].apply(
        lambda x: True if any(keyword in ' '.join(x) for keyword in keywords) else False
    )

#### Creating columns of different house rules we want to study

In [42]:
# create a sorted list of unique house rules so we can see what are the key house rules we want to look at
unique_house_rules = sorted(list(airbnb['house_rules'].str.split(',').explode().unique()))
unique_house_rules

['',
 'Check out: 10:00 am',
 'Check out: 10:00 pm',
 'Check out: 11:00 am',
 'Check out: 11:00 pm',
 'Check out: 12:00 am',
 'Check out: 12:00 pm',
 'Check out: 1:00 am',
 'Check out: 1:00 pm',
 'Check out: 2:00 am',
 'Check out: 2:00 pm',
 'Check out: 3:00 am',
 'Check out: 3:00 pm',
 'Check out: 4:00 am',
 'Check out: 4:00 pm',
 'Check out: 5:00 pm',
 'Check out: 6:00 pm',
 'Check out: 7:00 am',
 'Check out: 7:00 pm',
 'Check out: 8:00 am',
 'Check out: 8:00 pm',
 'Check out: 9:00 am',
 'Check-in: 10:00 am - 10:00 pm',
 'Check-in: 10:00 am - 11:00 pm',
 'Check-in: 10:00 am - 12:00 am',
 'Check-in: 10:00 am - 12:00 pm',
 'Check-in: 10:00 am - 1:00 am',
 'Check-in: 10:00 am - 1:00 pm',
 'Check-in: 10:00 am - 2:00 pm',
 'Check-in: 10:00 am - 3:00 pm',
 'Check-in: 10:00 am - 4:00 pm',
 'Check-in: 10:00 am - 5:00 pm',
 'Check-in: 10:00 am - 6:00 pm',
 'Check-in: 10:00 am - 7:00 pm',
 'Check-in: 10:00 am - 8:00 pm',
 'Check-in: 10:00 am - 9:00 pm',
 'Check-in: 10:00 pm - 12:00 am',
 'Chec

In [43]:
# Next, we want to make another column that holds the lower case list of all house_rules
airbnb['house_rules_list'] = airbnb['house_rules'].apply(lambda x: [item.lower().strip() for item in x.split(',')])

# I'm adding keywords that could help us find the 'common amenities'
# each of these keys will be another column and the value will be either yes or no (depending if listing contains these keywords)
# please feel free to add more amenities categories
# we could also do the same thing for safety rules if we want to analyze that as well
categories = {
    'no_pets': ['no pets'],
    'no_smoking': ['no smoking'],
    'no_parties_or_events': ['no parties or events']
}

# Initialize columns with "no"
for category in categories:
    airbnb[category] = "no"

# Update columns with "yes" where the amenity is present
for category, keywords in categories.items():
    airbnb[category] = airbnb['house_rules_list'].apply(
        lambda x: True if any(keyword in x for keyword in keywords) else False)

#### Creating columns of different safety rules we want to study

In [44]:
# create a sorted list of unique safety rules so we can see what are the key safety rules we want to look at
unique_safety_rules = sorted(list(airbnb['safety_rules'].str.split(',').explode().unique()))
unique_safety_rules

# after looking at the output, suggest we do not include any of these safety rules as categories

['',
 ' other body of water',
 ' river',
 "Airbnb's COVID-19 safety practices apply",
 'Carbon monoxide alarm',
 'Carbon monoxide alarm not reported Show more',
 'Carbon monoxide detector not required Show more',
 'Climbing or play structure',
 'Heights without rails or protection',
 'May encounter potentially dangerous animals',
 'Nearby lake',
 'No carbon monoxide alarm',
 'No smoke alarm',
 'Not Available',
 'Pool/hot tub without a gate or lock',
 'Security camera/recording device Show more',
 'Show more',
 'Smoke alarm',
 'Smoke alarm not reported Show more',
 '\U000f1801',
 '\U000f1803',
 '\U000f1806',
 '\U000f9001']

#### creating columns that have the total number of house rules, image links, and amenities

In [45]:
# Next, we want to make another column that holds the lower case list of all image links
airbnb['img_links_list'] = airbnb['img_links'].apply(lambda x: [item.lower().strip() for item in x.split(' ')])

In [46]:
# create three columns, each having the total number of house rules, image links, and amenities
airbnb['house_rules_num'] = airbnb['house_rules_list'].apply(lambda x:len(x))
airbnb['img_links_num'] = airbnb['img_links_list'].apply(lambda x:len(x))
airbnb['amenities_num'] = airbnb['amenities_list'].apply(lambda x:len(x))


#### looking at checkin and checkout columns to see how they can be cleaned

In [47]:
airbnb.checkin.value_counts(dropna=False).reset_index().head(10)

,index,checkin
0,After 3 00 pm,2554
1,After 2 00 pm,1545
2,Flexible,926
3,<NA>,800
4,After 4 00 pm,650
5,After 12 00 pm,644
6,After 1 00 pm,531
7,3 00 pm - 10 00 pm,312
8,3 00 pm - 8 00 pm,268
9,3 00 pm - 9 00 pm,243


In [48]:
airbnb.checkout.value_counts(dropna=False).reset_index()

,index,checkout
0,11 00 am,4391
1,12 00 pm,2648
2,10 00 am,2622
3,NaN,2450
4,9 00 am,186
5,1 00 pm,178
6,12 00 am,82
7,2 00 pm,72
8,3 00 pm,50
9,8 00 am,30


#### Assumption 2: When there are no checkin or checkout time, then we will assume it's flexible

In [49]:
# fill na for checkin with Flexible
airbnb["checkin"].fillna("Flexible", inplace = True)
# fill na for checkout with Flexible
airbnb["checkout"].fillna("Flexible", inplace = True)

Check to see if we have any duplicated ids

In [50]:
airbnb[airbnb.duplicated(['id'], keep = False)] # no duplicated ids. 

,Unnamed: 0,id,name,rating,reviews,host_name,host_id,address,features,amenities,safety_rules,house_rules,img_links,price,country,bathrooms,beds,guests,toilets,bedrooms,studios,checkin,checkout,usd_price,amenities_list,tv,wifi,chromecast,refrigerator,kitchen,view,parking,workspace,pool,gym,house_rules_list,no_pets,no_smoking,no_parties_or_events,img_links_list,house_rules_num,img_links_num,amenities_num


In [51]:
airbnb.head()

,Unnamed: 0,id,name,rating,reviews,host_name,host_id,address,features,amenities,safety_rules,house_rules,img_links,price,country,bathrooms,beds,guests,toilets,bedrooms,studios,checkin,checkout,usd_price,amenities_list,tv,wifi,chromecast,refrigerator,kitchen,view,parking,workspace,pool,gym,house_rules_list,no_pets,no_smoking,no_parties_or_events,img_links_list,house_rules_num,img_links_num,amenities_num
0,0,49849504,Perla bungalov,4.71,64,Mehmetcan,357334205,"Kartepe, Kocaeli, Turkey","2 guests,2 bedrooms,1 bed,1 bathroom","Mountain view,Valley view,Lake access,Kitchen,...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: Flexible,Check out: 12:00 pm,Pets ar...",https://a0.muscache.com/im/pictures/a5da5cb7-c...,8078.00,Turkey,1,1,2,0,2,0,Flexible,12 00 pm,244.14,"[mountain view, valley view, lake access, kitc...",True,True,False,False,True,True,True,False,False,False,"[check-in: flexible, check out: 12:00 pm, pets...",False,False,False,[https://a0.muscache.com/im/pictures/a5da5cb7-...,5,6,10
1,1,50891766,Authentic Beach Architect Sheltered Villa with...,-1.00,0,Fatih,386223873,"Kaş, Antalya, Turkey","4 guests,2 bedrooms,2 beds,2 bathrooms","Kitchen,Wifi,Dedicated workspace,Free parking ...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: 4:00 pm - 11:00 pm,Check out: 10:00 ...",https://a0.muscache.com/im/pictures/61b70855-2...,4665.00,Turkey,2,2,4,0,2,0,4 00 pm - 11 00 pm,10 00 am,140.99,"[kitchen, wifi, dedicated workspace, free park...",True,True,False,False,True,False,True,True,True,False,"[check-in: 4:00 pm - 11:00 pm, check out: 10:0...",True,True,True,[https://a0.muscache.com/im/pictures/61b70855-...,5,6,10
2,2,50699164,cottages sataplia,4.85,68,Giorgi,409690853,"Imereti, Georgia","4 guests,1 bedroom,3 beds,1 bathroom","Mountain view,Kitchen,Wifi,Dedicated workspace...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 1:00 pm,Check out: 12:00 pm,Se...",https://a0.muscache.com/im/pictures/miso/Hosti...,5991.00,Georgia,1,3,4,0,1,0,After 1 00 pm,12 00 pm,2202.57,"[mountain view, kitchen, wifi, dedicated works...",True,True,False,False,True,True,True,True,False,False,"[check-in: after 1:00 pm, check out: 12:00 pm,...",False,False,False,[https://a0.muscache.com/im/pictures/miso/host...,4,6,10
3,3,49871422,Sapanca Breathable Bungalow,5.00,13,Melih,401873242,"Sapanca, Sakarya, Turkey","4 guests,1 bedroom,2 beds,1 bathroom","Mountain view,Valley view,Kitchen,Wifi,Free pa...","󹀁,Airbnb's COVID-19 safety practices apply,󱠃,N...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/72e6396e-e...,11339.00,Turkey,1,2,4,0,1,0,After 2 00 pm,12 00 pm,342.70,"[mountain view, valley view, kitchen, wifi, fr...",True,True,False,False,True,True,True,False,True,False,"[check-in: after 2:00 pm, check out: 12:00 pm,...",True,True,False,[https://a0.muscache.com/im/pictures/72e6396e-...,4,6,10
4,4,51245886,Bungalov Ev 2,-1.00,0,Arp Sapanca,414884116,"Sapanca, Sakarya, Turkey","2 guests,1 bedroom,1 bed,1 bathroom","Kitchen,Wifi,Free parking on premises,TV,Air c...","󹀁,Airbnb's COVID-19 safety practices apply,󱠆,C...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",https://a0.muscache.com/im/pictures/73973308-e...,6673.00,Turkey,1,1,2,0,1,0,After 2 00 pm,12 00 pm,201.68,"[kitchen, wifi, free parking on premises, tv, ...",True,True,False,False,True,False,True,False,False,False,"[check-in: after 2:00 pm, check out: 12:00 pm,...",True,True,True,[https://a0.muscache.com/im/pictures/73973308-...,8,6,8


#### Drop unnecessary columns

In [52]:
# Drop unnecessary columns Unnamed: 0, host_name,	address, features,	amenities, amenities_list, house_rules, price (because we have new column called usd_price)
airbnb.drop(['Unnamed: 0', 'host_name', 'features',	'amenities', 'safety_rules', 'amenities_list', 'house_rules','house_rules_list', 'safety_rules','img_links_list','img_links'], axis=1, inplace=True)

In [53]:
# final dataframe to analyze
airbnb.head()

,id,name,rating,reviews,host_id,address,price,country,bathrooms,beds,guests,toilets,bedrooms,studios,checkin,checkout,usd_price,tv,wifi,chromecast,refrigerator,kitchen,view,parking,workspace,pool,gym,no_pets,no_smoking,no_parties_or_events,house_rules_num,img_links_num,amenities_num
0,49849504,Perla bungalov,4.71,64,357334205,"Kartepe, Kocaeli, Turkey",8078.00,Turkey,1,1,2,0,2,0,Flexible,12 00 pm,244.14,True,True,False,False,True,True,True,False,False,False,False,False,False,5,6,10
1,50891766,Authentic Beach Architect Sheltered Villa with...,-1.00,0,386223873,"Kaş, Antalya, Turkey",4665.00,Turkey,2,2,4,0,2,0,4 00 pm - 11 00 pm,10 00 am,140.99,True,True,False,False,True,False,True,True,True,False,True,True,True,5,6,10
2,50699164,cottages sataplia,4.85,68,409690853,"Imereti, Georgia",5991.00,Georgia,1,3,4,0,1,0,After 1 00 pm,12 00 pm,2202.57,True,True,False,False,True,True,True,True,False,False,False,False,False,4,6,10
3,49871422,Sapanca Breathable Bungalow,5.00,13,401873242,"Sapanca, Sakarya, Turkey",11339.00,Turkey,1,2,4,0,1,0,After 2 00 pm,12 00 pm,342.70,True,True,False,False,True,True,True,False,True,False,True,True,False,4,6,10
4,51245886,Bungalov Ev 2,-1.00,0,414884116,"Sapanca, Sakarya, Turkey",6673.00,Turkey,1,1,2,0,1,0,After 2 00 pm,12 00 pm,201.68,True,True,False,False,True,False,True,False,False,False,True,True,True,8,6,8


In [54]:
airbnb.to_csv('airbnb_clean_2.csv')